## imports, etc.

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
from tqdm.notebook import tqdm
import os
from sklearn.semi_supervised import LabelPropagation
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import fetch_openml
import matplotlib.pyplot as plt

In [3]:
Xv_train = torch.load("data/FashionMNIST_train_latent.pt")
Xv_test = torch.load("data/FashionMNIST_test_latent.pt")
Xv = torch.cat((Xv_train, Xv_test), 0)
y_train = torch.load("data/FashionMNIST_train_labels.pt")
y_test = torch.load("data/FashionMNIST_test_labels.pt")
y = torch.cat((y_train, y_test), 0)

In [4]:
Xa_train = torch.load("data/AE_train.pt")
Xa_test = torch.load("data/AE_test.pt")
Xa = torch.cat((Xa_train, Xa_test), 0).detach().cpu()

In [5]:
os.chdir("../fashion_ml")

In [6]:
from fashion_vae import FashionVAE
from data_loader import EZ_Dataloader
from experiment import FashionML

In [7]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
#np.random.seed(0)

## limit data labels

discard all but 200 random labels (we can increase the number but I feel like 1000+ doesn't really count as semi-supervised anymore)

In [8]:
n_labeled = 200

In [9]:
ind = np.random.randint(len(Xv), size=(n_labeled))
y_lim = np.repeat(-1, len(y))
y_lim[ind] = y[ind]

## compare: clustering

VAE - search for optimal k: 2 is optimal, anything over 3 predicts everything as the first class

In [10]:
def test_knn(X, y, n):
    lpm = LabelPropagation(kernel='knn',n_neighbors=n,n_jobs=-1,max_iter=5000)
    lpm.fit(X, y_lim)
    print(lpm.score(X, y))
    y_pred = lpm.predict(X)
    print(np.unique(y_pred))
    return lpm

In [11]:
test_knn(Xv, y, 2)

0.1028
[0 1 2 3 4 5 6 7 8 9]


LabelPropagation(kernel='knn', max_iter=5000, n_jobs=-1, n_neighbors=2)

In [12]:
test_knn(Xv, y, 3)

0.1013
[0 1 2 3 4 5 6 7 8 9]


LabelPropagation(kernel='knn', max_iter=5000, n_jobs=-1, n_neighbors=3)

In [13]:
test_knn(Xv, y, 7)

0.1
[0]


LabelPropagation(kernel='knn', max_iter=5000, n_jobs=-1)

AE - search for optimal k: 2 still best. and accuracy is slightly better than for vae? 

In [12]:
test_knn(Xa, y, 2)

0.10335714285714286
[0 1 2 3 4 5 6 7 8 9]


LabelPropagation(kernel='knn', max_iter=5000, n_jobs=-1, n_neighbors=2)

In [13]:
test_knn(Xa, y, 3)

0.10194285714285714
[0 1 2 3 4 5 6 7 8 9]


LabelPropagation(kernel='knn', max_iter=5000, n_jobs=-1, n_neighbors=3)

In [14]:
test_knn(Xa, y, 7)

0.1
[0]


LabelPropagation(kernel='knn', max_iter=5000, n_jobs=-1)

## compare: mlp

test a few networks on vae latent representations

In [13]:
Xv_train = Xv[y_lim != -1]
Xv_test = Xv[y_lim == -1]
y_train = y[y_lim != -1]
y_test = y[y_lim == -1]

In [18]:
clf = MLPClassifier(hidden_layer_sizes=(100,20,5), tol=1e-4)
clf.fit(Xv_train, y_train)

MLPClassifier(hidden_layer_sizes=(100, 20, 5))

In [19]:
clf.score(Xv_test, y_test)

0.4000859598853868

In [22]:
clf = MLPClassifier(hidden_layer_sizes=(200,100,25,5), tol=1e-5)
clf.fit(Xv_train, y_train)

MLPClassifier(hidden_layer_sizes=(200, 100, 25, 5), tol=1e-05)

In [23]:
clf.score(Xv_test, y_test)

0.479054441260745

test same networks on vanilla ae

In [15]:
Xa_train = Xa[y_lim != -1]
Xa_test = Xa[y_lim == -1]
y_train = y[y_lim != -1]
y_test = y[y_lim == -1]

In [16]:
clf = MLPClassifier(hidden_layer_sizes=(100,20,5), tol=1e-4)
clf.fit(Xa_train, y_train)

MLPClassifier(hidden_layer_sizes=(100, 20, 5))

In [18]:
clf.score(Xa_test, y_test)

0.38530085959885385

In [21]:
clf = MLPClassifier(hidden_layer_sizes=(200,100,25,5), tol=1e-5)
clf.fit(Xa_train, y_train)

MLPClassifier(hidden_layer_sizes=(200, 100, 25, 5), tol=1e-05)

In [22]:
clf.score(Xa_test, y_test)

0.531704871060172